# PCA

varianza en la proyección de datos, varianza se captura mejor con vectores que siguen mejor la tendencia de los datos covarianza. vector arbitrario de dirección por matriz de covarianza. Con valores propios en la diagonal de la matriz de covariannza. Vectores propios son las componentes prinncipales. Reducir a menos variables, estando dispuesto a sacrificar varianza, para dar al modelo de ML menos cantidad de datos

# Index

1. [Packages](#packages)
2. [Visualizaciones](#visualizaciones)
3. [Busco anomalías](#busco-anomalías)

# Packages

In [ ]:
import sys
import os
from pathlib import Path
sys.path.append(os.path.abspath(".."))

from mlparadetectarfraudes.data import data

import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

#data_estandarizados = pd.read_csv(data_interim_dir("data_estandarizados.csv"))


In [ ]:
#Estandarizamos los datos previo a hacer el PCA
scaler = StandardScaler()
data_estandarizados = scaler.fit_transform(data)




In [ ]:
# Aplicar PCA
pca = PCA(n_components=0.95)  # Mantener el 95% de varianza
X_pca = pca.fit_transform(X_processed_df)

# Visualizar varianza explicada
plt.figure(figsize=(10, 6))
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Número de componentes')
plt.ylabel('Varianza explicada acumulativa')
plt.title('PCA - Varianza explicada por componentes')
plt.grid(True)
plt.show()

# Crear DataFrame con componentes principales y variable objetivo
pca_df = pd.DataFrame(X_pca, columns=[f'PC{i+1}' for i in range(X_pca.shape[1])])
pca_df['is_fraud'] = y.values

# Visualizar primeros dos componentes coloreados por fraude
plt.figure(figsize=(10, 8))
fraud_data = pca_df[pca_df['is_fraud'] == 1]
normal_data = pca_df[pca_df['is_fraud'] == 0]
plt.scatter(normal_data['PC1'], normal_data['PC2'], alpha=0.5, label='Normal', s=10)
plt.scatter(fraud_data['PC1'], fraud_data['PC2'], alpha=0.8, label='Fraude', s=20, color='red')
plt.xlabel('Primer Componente Principal (Explica {:.2f}% de varianza)'.format(pca.explained_variance_ratio_[0]*100))
plt.ylabel('Segunda Componente Principal (Explica {:.2f}% de varianza)'.format(pca.explained_variance_ratio_[1]*100))
plt.title('PCA - Visualización de Transacciones Normales y Fraudulentas')
plt.legend()
plt.show()

print("Interpretación:")
print("- Si los puntos rojos (fraudes) están agrupados en áreas específicas, indica que el PCA puede ayudar a detectar patrones de fraude")
print("- Si están dispersos por todo el gráfico, significa que los fraudes no siguen un patrón claro en las dos primeras componentes")
print("- Los outliers (puntos alejados del grupo principal) podrían representar transacciones anómalas")

In [ ]:
# Usar PCA con el número de componentes que explican el 95% de varianza
pca_95 = PCA(n_components=0.95)
X_pca_95 = pca_95.fit_transform(X_processed_df)

print(f"Número de componentes para 95% de varianza: {pca_95.n_components_}")
print(f"Varianza explicada acumulada: {np.sum(pca_95.explained_variance_ratio_):.4f}")

# Visualizar varianza explicada por componente
plt.figure(figsize=(10, 6))
plt.bar(range(1, len(pca_95.explained_variance_ratio_)+1), pca_95.explained_variance_ratio_)
plt.xlabel('Componente Principal')
plt.ylabel('Varianza Explicada')
plt.title('Varianza Explicada por Cada Componente Principal')
plt.show()

Vemos la cantidad de variabilidad explicada de cada componente y hacemos un dataframe con los componentes que nos interesan

In [ ]:
# Aplicar PCA
pca = PCA(n_components=3) #reducimos a 3 componentes
componentes = pca.fit_transform(data_estandarizados)

#Creamos dataframe con los componentes principales
pca_df = pd.DataFrame(data=componentes, columns=['PC1', 'PC2', 'PC3'], index=data.index)

# Visualizaciones

In [ ]:
#Visualizamos la varinza explicada
plt.figure(figsize=(10, 6))
plt.bar(range(1,4), pca.explained_variance_ratio_, alpha=0.6)
plt.plot(range(1,4), np.cumsum(pca.explained_variance_ratio_), 'ro-')
plt.xlabel('Componentes principales')
plt.ylabel('Varianza explicada')
plt.title('Varianza explicada por cada componente principal')
plt.xticks(range(1,4))
plt.grid(True)
plt.show()

In [ ]:
# Visualización 3D de los componentes principales
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(pca_df['PC1'], pca_df['PC2'], pca_df['PC3'], alpha=0.5)
ax.set_xlabel('PC1 (%.2f%%)' % (pca.explained_variance_ratio_[0]*100))
ax.set_ylabel('PC2(%.2f%%)' % (pca.explained_variance_ratio_[1]*100))
ax.set_zlabel('PC3(%.2f%%)' % (pca.explained_variance_ratio_[2]*100))
ax.set_title('Visualizaciónn 3D de Componentes Principales')
plt.show()

# Busco anomalías

In [ ]:
# Reconstruir los datos a partir de los componentes principales
reconstruidos = pca.inverse_transform(componentes)
#calcular el error de reconstrucción MSE
mse = np.mean(np.square(retornos_estandarizados - reconstruidos), axis=1)

#Identificar anomalías (umbral = percentil 95)
umbral = np.percentile(mse, 95)
anomalias = mse > umbral

In [ ]:
#Visualizar anomalías
plt.figure(figsize=(12, 6))
plt.plot(retornos.index, mse, label='Error de reconstrucción')
plt.axhline(y=umbral, color='r', linestyle='--', label='Umbral')

In [ ]:
Visualizar anomalias
plt.figure(figsize=(12, 6))
plt.plot(retornos.index, mse, label='Error de reconstrucción')
plt.scatter(retornos.index[anomalias], mse[anomalias], color='r', label='Anomalías')
plt.axhline(y=umbral, color='r', linestyle='--', label='Anomalías')
plt.title('Detección de Anomalías en mercados financieros')
plt.xlabel('Fecha')
plt.ylabel('Error de reconstrucción')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Umbral es la linea de rayitas, puntos rojos son las anomalias, una anomalía tan grande me arruina la visualización de las otras. Corona virus puede ser anomaias

In [ ]:
#Analizamos fechas con anomalías
anomalias_df = retornos[anomalias]
print("Fechas con anomalías:")
print(anomalias_df.index)

In [ ]:
#Vemos los 5 días con mayores anomalías
top_anomalias = anomalias_df.iloc[np.argsort(-mse[anomalias])[:5]]
print("/nTop 5 días con mayores anomalias:")
print(top_anomalias)